<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [203]:
!pip install wrds datetime
import pandas as pd, numpy as np, wrds, pytz
import datetime as dt

# Read datasets

In [204]:
ticker = pd.DataFrame()
for i in range(2019,2020,1):
    read = pd.read_parquet(f'/content/ticker_{i}.parquet.gzip')
    ticker = ticker.append(read)
col = ['date_utc','num_comments','score',
        'ticker_a','ticker_b']
ticker = ticker[col]
### change directory on WRDS

# Combine tickers

In [205]:
def comb_tic(x):
    if len(x[1]) == 0:
        return x[0]
    elif len(x[0]) == 0:
        return x[1]
    else:
       return list(set(x[0]).union(x[1]))

In [206]:
ticker['comb_tic'] = ticker[['ticker_a','ticker_b']].apply(comb_tic, axis=1)

In [207]:
ticker['add_col'] = ticker['comb_tic'].apply(len)

# Separate columns

In [208]:
pos = len(ticker.columns)
tics = []
for i in range(pos, ticker['add_col'].max() + pos, 1):
    tics = tics + [f'tic{i - pos}']

In [209]:
ticker[tics] = pd.DataFrame(ticker['comb_tic'].to_list(), index=ticker.index)
ticker.drop(['ticker_a','ticker_b','comb_tic'],inplace=True, axis=1)

# Merge rewards from csv

In [210]:
wsb = pd.read_csv('/content/r_wallstreetbets_posts.csv',index_col='id',low_memory=False)

In [211]:
wsb = wsb[['total_awards_received']]

In [212]:
ticker = ticker.merge(wsb,how='left',left_index=True,right_index=True)

# Shift dates

In [213]:
ticker['datetime'] = ticker['date_utc'].dt.tz_convert('US/Eastern')

In [223]:
start = dt.time(16,0,0)
end = dt.time(23,59,59)
ticker['date'] = np.where(ticker['datetime'].dt.time.between(start,end),
                          ticker['datetime'].dt.date + pd.DateOffset(days=1),
                          ticker['datetime'].dt.date)

In [231]:
ticker['date'] = pd.to_datetime(ticker['date'],unit='ms')

In [ ]:
ticker['dat']

# Transpose

In [128]:
tic_t = pd.wide_to_long(ticker.reset_index(),'tic',i='id',j='num')[lambda x: ~x['tic'].isna()]

In [129]:
tic_t.head()

,,num_comments,add_col,total_awards_received,date_utc,score,tic
id,num,,,,,,
eiaz8e,0,5,4,0.0,2019-12-31 23:58:26+00:00,1,DD
eiatp9,0,38,2,0.0,2019-12-31 23:45:57+00:00,1,DD
eiasq6,0,0,1,0.0,2019-12-31 23:43:38+00:00,1,BSGM
eial7r,0,16,2,0.0,2019-12-31 23:26:59+00:00,1,PE
eia4wt,0,0,5,0.0,2019-12-31 22:50:25+00:00,1,PTI
